In [1]:
from pyperplan import _parse, _ground
import numpy as np
from copy import deepcopy
from abc import ABC, abstractmethod
from embedding import NaiveEmb

In [2]:
problem = _parse(domain_file='pddl_files/modded_transport/6/domain.pddl',
                 problem_file='pddl_files/modded_transport/6/p01.pddl')
task = _ground(problem)

In [3]:
len(task.facts)

884

### Binary embedding 

In [5]:
E = NaiveEmb(task)

In [9]:
print(E.state_to_emb(task.initial_state).shape)
print(E.state_to_emb(task.initial_state).dtype)
print(E.state_to_emb(task.initial_state)[:10])

(884,)
bool
[False False False False False False False False False False]


In [21]:
# NOTE THAT LOTS OF MUTEXES VIOLATED BY THIS IN TRANSPORT!!! WE _REALLY_ NEED SAS+
random_emb = np.random.randint(0, 2, size=len(task.facts), dtype=np.bool)
E.emb_to_state(random_emb) 

frozenset({'(at package-1 city-loc-11)',
           '(at package-1 city-loc-14)',
           '(at package-1 city-loc-15)',
           '(at package-1 city-loc-17)',
           '(at package-1 city-loc-18)',
           '(at package-1 city-loc-20)',
           '(at package-1 city-loc-24)',
           '(at package-1 city-loc-26)',
           '(at package-1 city-loc-27)',
           '(at package-1 city-loc-28)',
           '(at package-1 city-loc-29)',
           '(at package-1 city-loc-30)',
           '(at package-1 city-loc-32)',
           '(at package-1 city-loc-35)',
           '(at package-1 city-loc-36)',
           '(at package-1 city-loc-38)',
           '(at package-1 city-loc-39)',
           '(at package-1 city-loc-40)',
           '(at package-1 city-loc-5)',
           '(at package-1 city-loc-8)',
           '(at package-10 city-loc-12)',
           '(at package-10 city-loc-14)',
           '(at package-10 city-loc-15)',
           '(at package-10 city-loc-17)',
           '(a

In [23]:
problem.objects

{'city-loc-1': location,
 'city-loc-2': location,
 'city-loc-3': location,
 'city-loc-4': location,
 'city-loc-5': location,
 'city-loc-6': location,
 'city-loc-7': location,
 'city-loc-8': location,
 'city-loc-9': location,
 'city-loc-10': location,
 'city-loc-11': location,
 'city-loc-12': location,
 'city-loc-13': location,
 'city-loc-14': location,
 'city-loc-15': location,
 'city-loc-16': location,
 'city-loc-17': location,
 'city-loc-18': location,
 'city-loc-19': location,
 'city-loc-20': location,
 'city-loc-21': location,
 'city-loc-22': location,
 'city-loc-23': location,
 'city-loc-24': location,
 'city-loc-25': location,
 'city-loc-26': location,
 'city-loc-27': location,
 'city-loc-28': location,
 'city-loc-29': location,
 'city-loc-30': location,
 'city-loc-31': location,
 'city-loc-32': location,
 'city-loc-33': location,
 'city-loc-34': location,
 'city-loc-35': location,
 'city-loc-36': location,
 'city-loc-37': location,
 'city-loc-38': location,
 'city-loc-39': locat

In [17]:
naive_emb = create_naive_embedding_fn(task.facts)

NameError: name 'create_naive_embedding_fn' is not defined

In [10]:
# So the binary embedding of the initial state is:
naive_emb(task.initial_state)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [7]:
# We can now get an N-dimensional continuous embedding using a trainable embedding matrix
# This is equivalent to having an N-dimensional embedding for each "fact/fluent", and then adding them together.
N = 16
W_emb = np.random.normal(0., 1e-2, (len(task.facts), N))

# So now the initial embedding of the initial state is:
np.expand_dims(naive_emb(task.initial_state), 0).dot(W_emb)

array([[-0.05030386, -0.01080087, -0.01502595,  0.04162736,  0.07952476,
         0.0474224 , -0.0305328 ,  0.04548838,  0.06364573,  0.0182024 ,
        -0.04808633,  0.00967416, -0.05022812, -0.01181706,  0.07991284,
         0.03560514]])

## Smarter embedding

- this is non-trivial... we should discuss